In [5]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import numpy as np
import time

begin = time.strftime('%Y-%m-%d_%H-%M-%S')

LOGDIR = './save'

tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

start_learning_rate = 0.5e-3  # 1e-3
adjust_learning_rate = 1e-5

onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=4)

loss = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

loss_val = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss_val = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
# train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
# tf.scalar_summary("loss", loss)
tf.summary.scalar("loss", loss)

tf.summary.scalar("loss_val", loss_val)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)
# saver.restore(sess, "save/model.ckpt")

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 10   # 13, 8, 12, 20
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        # train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
        loss_value = loss.eval(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0})
        # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
        print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        if i % 10 == 0:
            xs_val, ys_val = driving_data.LoadValBatch(batch_size)
            # xs, ys = driving_data.LoadTrainBatch(batch_size)
            loss_val = loss.eval(feed_dict={model.x:xs_val, model.y_: ys_val, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss_val: %g" % (epoch, i, loss_val))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)


correct_prediction = tf.equal(tf.argmax(onehot_labels, 1), tf.argmax(model.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Train Accuracy:', sess.run(accuracy, feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}))
print('Validation Accuracy:', sess.run(accuracy, feed_dict={model.x: xs_val, model.y_: ys_val, model.keep_prob: 1.0}))

end = time.strftime('%Y-%m-%d_%H-%M-%S')
print('begin: ', begin)
print('end: ', end)

print("Run the command line:\n",
      "--> tensorboard --logdir=./logs --port=6006",
      "\nThen open http://0.0.0.0:6006/ into your web browser")

# os.system("sudo -s shutdown -h now")


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch: 0, Step: 0, Loss: 1.3884
Epoch: 0, Step: 0, Loss_val: 1.3894
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 1, Loss: 1.38616
Epoch: 0, Step: 2, Loss: 1.3792
Epoch: 0, Step: 3, Loss: 1.37819
Epoch: 0, Step: 4, Loss: 1.37053
Epoch: 0, Step: 5, Loss: 1.36695
Epoch: 0, Step: 6, Loss: 1.35438
Epoch: 0, Step: 7, Loss: 1.34396
Epoch: 0, Step: 8, Loss: 1.35892
Epoch: 0, Step: 9, Loss: 1.28696
Epoch: 0, Step: 10, Loss: 1.30104
Epoch: 0, Step: 10, Loss_val: 1.32264
Epoch: 0, Step: 11, Loss: 1.33417
Epoch: 0, Step: 12, Loss: 1.33158
Epoch: 0, Step: 13, Loss: 1.35693
Epoch: 0, Step: 14, Loss: 1.32509
Epoch: 0, Step: 15, Loss: 1.32348
Epoch: 0, Step: 16, Loss: 1.34716
Epoch: 0, Step: 17, Loss: 1.3003
Epoch: 0, Step: 18, Loss: 1.33961
Epoch: 0, Step: 19, Loss: 1.28917
Epoch: 0, Step: 20, Loss: 1.28463
Epoch: 0, Step: 20, Loss_val: 1.35396
Epoch: 0, Step: 21, Loss: 1.29998
Epoch: 0, Step: 22, Loss: 1.29691
Epoch: 0, Step: 23, Loss: 1.3119
Epoch: 0, Step: 24, Loss: 1.34299
Epoch: 0, Ste

Epoch: 1, Step: 87, Loss: 0.435343
Epoch: 1, Step: 88, Loss: 0.342644
Epoch: 1, Step: 89, Loss: 0.287196
Epoch: 1, Step: 90, Loss: 0.377136
Epoch: 1, Step: 90, Loss_val: 0.407605
Epoch: 1, Step: 91, Loss: 0.30442
Epoch: 1, Step: 92, Loss: 0.387717
Epoch: 1, Step: 93, Loss: 0.277182
Epoch: 1, Step: 94, Loss: 0.217385
Epoch: 1, Step: 95, Loss: 0.380045
Epoch: 1, Step: 96, Loss: 0.286989
Epoch: 1, Step: 97, Loss: 0.48631
Epoch: 1, Step: 98, Loss: 0.315135
Epoch: 1, Step: 99, Loss: 0.45993
Epoch: 1, Step: 100, Loss: 0.421608
Epoch: 1, Step: 100, Loss_val: 0.307068
Model saved in file: ./save/model.ckpt
Epoch: 1, Step: 101, Loss: 0.1997
Epoch: 1, Step: 102, Loss: 0.424586
Epoch: 1, Step: 103, Loss: 0.286165
Epoch: 1, Step: 104, Loss: 0.298654
Epoch: 1, Step: 105, Loss: 0.282516
Epoch: 1, Step: 106, Loss: 0.369953
Epoch: 1, Step: 107, Loss: 0.311737
Epoch: 1, Step: 108, Loss: 0.293561
Epoch: 1, Step: 109, Loss: 0.321361
Epoch: 1, Step: 110, Loss: 0.283878
Epoch: 1, Step: 110, Loss_val: 0.308

Epoch: 3, Step: 48, Loss: 0.336348
Epoch: 3, Step: 49, Loss: 0.183886
Epoch: 3, Step: 50, Loss: 0.283659
Epoch: 3, Step: 50, Loss_val: 0.252881
Epoch: 3, Step: 51, Loss: 0.187482
Epoch: 3, Step: 52, Loss: 0.231374
Epoch: 3, Step: 53, Loss: 0.251338
Epoch: 3, Step: 54, Loss: 0.205388
Epoch: 3, Step: 55, Loss: 0.189683
Epoch: 3, Step: 56, Loss: 0.148087
Epoch: 3, Step: 57, Loss: 0.222872
Epoch: 3, Step: 58, Loss: 0.331887
Epoch: 3, Step: 59, Loss: 0.158659
Epoch: 3, Step: 60, Loss: 0.149003
Epoch: 3, Step: 60, Loss_val: 0.170093
Epoch: 3, Step: 61, Loss: 0.159566
Epoch: 3, Step: 62, Loss: 0.201373
Epoch: 3, Step: 63, Loss: 0.225376
Epoch: 3, Step: 64, Loss: 0.25723
Epoch: 3, Step: 65, Loss: 0.200321
Epoch: 3, Step: 66, Loss: 0.162086
Epoch: 3, Step: 67, Loss: 0.239822
Epoch: 3, Step: 68, Loss: 0.18015
Epoch: 3, Step: 69, Loss: 0.225643
Epoch: 3, Step: 70, Loss: 0.223773
Epoch: 3, Step: 70, Loss_val: 0.419738
Epoch: 3, Step: 71, Loss: 0.229878
Epoch: 3, Step: 72, Loss: 0.426524
Epoch: 3, 

Epoch: 5, Step: 8, Loss: 0.221749
Epoch: 5, Step: 9, Loss: 0.278473
Epoch: 5, Step: 10, Loss: 0.212236
Epoch: 5, Step: 10, Loss_val: 0.22305
Epoch: 5, Step: 11, Loss: 0.136723
Epoch: 5, Step: 12, Loss: 0.167836
Epoch: 5, Step: 13, Loss: 0.138761
Epoch: 5, Step: 14, Loss: 0.188717
Epoch: 5, Step: 15, Loss: 0.175943
Epoch: 5, Step: 16, Loss: 0.192167
Epoch: 5, Step: 17, Loss: 0.215305
Epoch: 5, Step: 18, Loss: 0.170386
Epoch: 5, Step: 19, Loss: 0.183588
Epoch: 5, Step: 20, Loss: 0.12402
Epoch: 5, Step: 20, Loss_val: 0.175217
Epoch: 5, Step: 21, Loss: 0.214125
Epoch: 5, Step: 22, Loss: 0.195827
Epoch: 5, Step: 23, Loss: 0.448462
Epoch: 5, Step: 24, Loss: 0.203679
Epoch: 5, Step: 25, Loss: 0.208347
Epoch: 5, Step: 26, Loss: 0.20469
Epoch: 5, Step: 27, Loss: 0.20295
Epoch: 5, Step: 28, Loss: 0.163095
Epoch: 5, Step: 29, Loss: 0.160866
Epoch: 5, Step: 30, Loss: 0.197811
Epoch: 5, Step: 30, Loss_val: 0.244833
Epoch: 5, Step: 31, Loss: 0.133194
Epoch: 5, Step: 32, Loss: 0.213878
Epoch: 5, Step

...

Epoch: 92, Step: 108, Loss: 0.00436293
Epoch: 92, Step: 109, Loss: 0.0635896
Epoch: 92, Step: 110, Loss: 0.0380986
Epoch: 92, Step: 110, Loss_val: 0.247748
Epoch: 92, Step: 111, Loss: 0.0157552
Epoch: 92, Step: 112, Loss: 0.0154991
Epoch: 92, Step: 113, Loss: 0.0388287
Epoch: 92, Step: 114, Loss: 0.0100621
Epoch: 92, Step: 115, Loss: 0.04383
Epoch: 92, Step: 116, Loss: 0.0681795
Epoch: 92, Step: 117, Loss: 0.0107886
Epoch: 92, Step: 118, Loss: 0.0661668
Epoch: 92, Step: 119, Loss: 0.0700221
Epoch: 92, Step: 120, Loss: 0.0252872
Epoch: 92, Step: 120, Loss_val: 0.322575
Epoch: 92, Step: 121, Loss: 0.0121078
Epoch: 92, Step: 122, Loss: 0.0196962
Epoch: 93, Step: 0, Loss: 0.0415813
Epoch: 93, Step: 0, Loss_val: 0.256875
Model saved in file: ./save/model.ckpt
Epoch: 93, Step: 1, Loss: 0.0651947
Epoch: 93, Step: 2, Loss: 0.0109484
Epoch: 93, Step: 3, Loss: 0.0144738
Epoch: 93, Step: 4, Loss: 0.0357849
Epoch: 93, Step: 5, Loss: 0.0151414
Epoch: 93, Step: 6, Loss: 0.0231768
Epoch: 93, Step: 7,

Epoch: 94, Step: 56, Loss: 0.000703136
Epoch: 94, Step: 57, Loss: 0.000721126
Epoch: 94, Step: 58, Loss: 0.00819657
Epoch: 94, Step: 59, Loss: 0.000925694
Epoch: 94, Step: 60, Loss: 0.0153701
Epoch: 94, Step: 60, Loss_val: 0.550784
Epoch: 94, Step: 61, Loss: 0.00072542
Epoch: 94, Step: 62, Loss: 0.000830094
Epoch: 94, Step: 63, Loss: 0.00178417
Epoch: 94, Step: 64, Loss: 0.00459233
Epoch: 94, Step: 65, Loss: 0.0014644
Epoch: 94, Step: 66, Loss: 0.000892425
Epoch: 94, Step: 67, Loss: 0.00249301
Epoch: 94, Step: 68, Loss: 0.00115877
Epoch: 94, Step: 69, Loss: 0.00334557
Epoch: 94, Step: 70, Loss: 0.00776793
Epoch: 94, Step: 70, Loss_val: 0.35655
Epoch: 94, Step: 71, Loss: 0.00153906
Epoch: 94, Step: 72, Loss: 0.00552289
Epoch: 94, Step: 73, Loss: 0.000678282
Epoch: 94, Step: 74, Loss: 0.00157932
Epoch: 94, Step: 75, Loss: 0.021935
Epoch: 94, Step: 76, Loss: 0.00618591
Epoch: 94, Step: 77, Loss: 0.00111493
Epoch: 94, Step: 78, Loss: 0.00387973
Epoch: 94, Step: 79, Loss: 0.000960696
Epoch:

Epoch: 96, Step: 0, Loss: 5.19473e-05
Epoch: 96, Step: 0, Loss_val: 1.01331
Model saved in file: ./save/model.ckpt
Epoch: 96, Step: 1, Loss: 0.000943857
Epoch: 96, Step: 2, Loss: 0.000808134
Epoch: 96, Step: 3, Loss: 0.00271613
Epoch: 96, Step: 4, Loss: 0.00038871
Epoch: 96, Step: 5, Loss: 0.00163716
Epoch: 96, Step: 6, Loss: 0.00376704
Epoch: 96, Step: 7, Loss: 0.000182863
Epoch: 96, Step: 8, Loss: 0.00476601
Epoch: 96, Step: 9, Loss: 0.0109226
Epoch: 96, Step: 10, Loss: 0.00120686
Epoch: 96, Step: 10, Loss_val: 1.4236
Epoch: 96, Step: 11, Loss: 0.00121109
Epoch: 96, Step: 12, Loss: 0.0175839
Epoch: 96, Step: 13, Loss: 0.0290129
Epoch: 96, Step: 14, Loss: 0.000533892
Epoch: 96, Step: 15, Loss: 0.000644464
Epoch: 96, Step: 16, Loss: 0.00185118
Epoch: 96, Step: 17, Loss: 0.000862719
Epoch: 96, Step: 18, Loss: 0.000627964
Epoch: 96, Step: 19, Loss: 0.000661148
Epoch: 96, Step: 20, Loss: 0.000525489
Epoch: 96, Step: 20, Loss_val: 0.245635
Epoch: 96, Step: 21, Loss: 0.000918152
Epoch: 96, 

Epoch: 97, Step: 69, Loss: 0.00582046
Epoch: 97, Step: 70, Loss: 0.020076
Epoch: 97, Step: 70, Loss_val: 0.39082
Epoch: 97, Step: 71, Loss: 0.0351221
Epoch: 97, Step: 72, Loss: 0.0129588
Epoch: 97, Step: 73, Loss: 0.043732
Epoch: 97, Step: 74, Loss: 0.0067505
Epoch: 97, Step: 75, Loss: 0.0268865
Epoch: 97, Step: 76, Loss: 0.0281653
Epoch: 97, Step: 77, Loss: 0.013525
Epoch: 97, Step: 78, Loss: 0.00821034
Epoch: 97, Step: 79, Loss: 0.0104126
Epoch: 97, Step: 80, Loss: 0.016062
Epoch: 97, Step: 80, Loss_val: 0.625934
Epoch: 97, Step: 81, Loss: 0.0024424
Epoch: 97, Step: 82, Loss: 0.0120271
Epoch: 97, Step: 83, Loss: 0.0152422
Epoch: 97, Step: 84, Loss: 0.0114064
Epoch: 97, Step: 85, Loss: 0.0542772
Epoch: 97, Step: 86, Loss: 0.0090354
Epoch: 97, Step: 87, Loss: 0.0158328
Epoch: 97, Step: 88, Loss: 0.00564787
Epoch: 97, Step: 89, Loss: 0.0140893
Epoch: 97, Step: 90, Loss: 0.0176382
Epoch: 97, Step: 90, Loss_val: 0.432533
Epoch: 97, Step: 91, Loss: 0.00262709
Epoch: 97, Step: 92, Loss: 0.0

Epoch: 99, Step: 14, Loss: 0.021162
Epoch: 99, Step: 15, Loss: 0.00154165
Epoch: 99, Step: 16, Loss: 0.000900828
Epoch: 99, Step: 17, Loss: 0.0055943
Epoch: 99, Step: 18, Loss: 0.000119852
Epoch: 99, Step: 19, Loss: 8.65277e-05
Epoch: 99, Step: 20, Loss: 0.00313395
Epoch: 99, Step: 20, Loss_val: 0.270803
Epoch: 99, Step: 21, Loss: 0.00228163
Epoch: 99, Step: 22, Loss: 0.000177527
Epoch: 99, Step: 23, Loss: 0.001818
Epoch: 99, Step: 24, Loss: 0.0246537
Epoch: 99, Step: 25, Loss: 1.8799e-05
Epoch: 99, Step: 26, Loss: 0.000147482
Epoch: 99, Step: 27, Loss: 0.00137383
Epoch: 99, Step: 28, Loss: 0.000564607
Epoch: 99, Step: 29, Loss: 0.00038695
Epoch: 99, Step: 30, Loss: 0.0196282
Epoch: 99, Step: 30, Loss_val: 0.366068
Epoch: 99, Step: 31, Loss: 0.00197877
Epoch: 99, Step: 32, Loss: 0.000238318
Epoch: 99, Step: 33, Loss: 0.000104164
Epoch: 99, Step: 34, Loss: 0.00121926
Epoch: 99, Step: 35, Loss: 0.000416776
Epoch: 99, Step: 36, Loss: 0.0103369
Epoch: 99, Step: 37, Loss: 0.00624037
Epoch: 

In [1]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import numpy as np
import time

begin = time.strftime('%Y-%m-%d_%H-%M-%S')

LOGDIR = './save'

tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

start_learning_rate = 0.5e-3  # 1e-3
adjust_learning_rate = 1e-5

onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=4)

loss = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

loss_val = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss_val = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
# train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
# tf.scalar_summary("loss", loss)
tf.summary.scalar("loss", loss)

tf.summary.scalar("loss_val", loss_val)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)
# saver.restore(sess, "save/model.ckpt")

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 10   # 13, 8, 12, 20
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        # train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
        loss_value = loss.eval(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0})
        # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
        print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        if i % 10 == 0:
            xs_val, ys_val = driving_data.LoadValBatch(batch_size)
            # xs, ys = driving_data.LoadTrainBatch(batch_size)
            loss_val = loss.eval(feed_dict={model.x:xs_val, model.y_: ys_val, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss_val: %g" % (epoch, i, loss_val))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)


correct_prediction = tf.equal(tf.argmax(onehot_labels, 1), tf.argmax(model.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Train Accuracy:', sess.run(accuracy, feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}))
print('Validation Accuracy:', sess.run(accuracy, feed_dict={model.x: xs_val, model.y_: ys_val, model.keep_prob: 1.0}))

end = time.strftime('%Y-%m-%d_%H-%M-%S')
print('begin: ', begin)
print('end: ', end)

print("Run the command line:\n",
      "--> tensorboard --logdir=./logs --port=6006",
      "\nThen open http://0.0.0.0:6006/ into your web browser")

# os.system("sudo -s shutdown -h now")




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
model read
Epoch: 0, Step: 0, Loss: 1.30769
Epoch: 0, Step: 0, Loss_val: 1.2944
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 1, Loss: 1.34093
Epoch: 0, Step: 2, Loss: 1.35513
Epoch: 0, Step: 3, Loss: 1.34923
Epoch: 0, Step: 4, Loss: 1.33144
Epoch: 0, Step: 5, Loss: 1.30499
Epoch: 0, Step: 6, Loss: 1.29786
Epoch: 0, Step: 7, Loss: 1.26874
Epoch: 0, Step: 8, Loss: 1.26811
Epoch: 0, Step: 9, Loss: 1.26658
Epoch: 0, Step: 10, Loss: 1.27268
Epoch: 0, Step: 10, Loss_val: 1.27669
Epoch: 0, Step: 11, Loss: 1.27862
Epoch: 0, Step: 12, Loss: 1.26313
Epoch: 0, Step: 13, Loss: 1.28847
Epoch: 0, Step: 14, Loss: 1.29563
Epoch: 0, Step: 15, Loss: 1.27351
Epoch: 0, Step: 16, Loss: 1.26787
Epoch: 0, Step: 17, Loss: 1.26906
Epoch: 0, Step: 18, Loss: 1.25677
Epoch: 0, Step: 19, Loss: 1.23412
Epoch: 0, Step: 20, Loss: 1.21167
Epoch: 0, Step: 20, Loss_val: 1.25969
Epoch: 0, Step: 2

Epoch: 0, Step: 195, Loss: 0.517766
Epoch: 0, Step: 196, Loss: 0.523436
Epoch: 0, Step: 197, Loss: 0.447781
Epoch: 0, Step: 198, Loss: 0.417991
Epoch: 0, Step: 199, Loss: 0.555822
Epoch: 0, Step: 200, Loss: 0.504074
Epoch: 0, Step: 200, Loss_val: 0.455778
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 201, Loss: 0.513241
Epoch: 0, Step: 202, Loss: 0.390707
Epoch: 1, Step: 0, Loss: 0.442154
Epoch: 1, Step: 0, Loss_val: 0.423798
Model saved in file: ./save/model.ckpt
Epoch: 1, Step: 1, Loss: 0.460843
Epoch: 1, Step: 2, Loss: 0.452141
Epoch: 1, Step: 3, Loss: 0.437242
Epoch: 1, Step: 4, Loss: 0.517961
Epoch: 1, Step: 5, Loss: 0.501957
Epoch: 1, Step: 6, Loss: 0.408068
Epoch: 1, Step: 7, Loss: 0.444912
Epoch: 1, Step: 8, Loss: 0.425593
Epoch: 1, Step: 9, Loss: 0.428691
Epoch: 1, Step: 10, Loss: 0.358202
Epoch: 1, Step: 10, Loss_val: 0.391598
Epoch: 1, Step: 11, Loss: 0.482222
Epoch: 1, Step: 12, Loss: 0.403083
Epoch: 1, Step: 13, Loss: 0.489062
Epoch: 1, Step: 14, Loss: 0.380148
Ep

Epoch: 1, Step: 199, Loss: 0.247158
Epoch: 1, Step: 200, Loss: 0.253637
Epoch: 1, Step: 200, Loss_val: 0.266841
Model saved in file: ./save/model.ckpt
Epoch: 1, Step: 201, Loss: 0.207579
Epoch: 1, Step: 202, Loss: 0.244313
Epoch: 2, Step: 0, Loss: 0.267524
Epoch: 2, Step: 0, Loss_val: 0.330257
Model saved in file: ./save/model.ckpt
Epoch: 2, Step: 1, Loss: 0.229107
Epoch: 2, Step: 2, Loss: 0.257044
Epoch: 2, Step: 3, Loss: 0.316093
Epoch: 2, Step: 4, Loss: 0.219719
Epoch: 2, Step: 5, Loss: 0.26635
Epoch: 2, Step: 6, Loss: 0.339498
Epoch: 2, Step: 7, Loss: 0.248012
Epoch: 2, Step: 8, Loss: 0.237439
Epoch: 2, Step: 9, Loss: 0.42811
Epoch: 2, Step: 10, Loss: 0.282535
Epoch: 2, Step: 10, Loss_val: 0.235
Epoch: 2, Step: 11, Loss: 0.195546
Epoch: 2, Step: 12, Loss: 0.235965
Epoch: 2, Step: 13, Loss: 0.272164
Epoch: 2, Step: 14, Loss: 0.258543
Epoch: 2, Step: 15, Loss: 0.238866
Epoch: 2, Step: 16, Loss: 0.274026
Epoch: 2, Step: 17, Loss: 0.190979
Epoch: 2, Step: 18, Loss: 0.270496
Epoch: 2, S

Epoch: 2, Step: 201, Loss: 0.252318
Epoch: 2, Step: 202, Loss: 0.191944
Epoch: 3, Step: 0, Loss: 0.25553
Epoch: 3, Step: 0, Loss_val: 0.298595
Model saved in file: ./save/model.ckpt
Epoch: 3, Step: 1, Loss: 0.224639
Epoch: 3, Step: 2, Loss: 0.227367
Epoch: 3, Step: 3, Loss: 0.227864
Epoch: 3, Step: 4, Loss: 0.144956
Epoch: 3, Step: 5, Loss: 0.276935
Epoch: 3, Step: 6, Loss: 0.225889
Epoch: 3, Step: 7, Loss: 0.177794
Epoch: 3, Step: 8, Loss: 0.196903
Epoch: 3, Step: 9, Loss: 0.184782
Epoch: 3, Step: 10, Loss: 0.147029
Epoch: 3, Step: 10, Loss_val: 0.320604
Epoch: 3, Step: 11, Loss: 0.178428
Epoch: 3, Step: 12, Loss: 0.29645
Epoch: 3, Step: 13, Loss: 0.226763
Epoch: 3, Step: 14, Loss: 0.169493
Epoch: 3, Step: 15, Loss: 0.246419
Epoch: 3, Step: 16, Loss: 0.336555
Epoch: 3, Step: 17, Loss: 0.180385
Epoch: 3, Step: 18, Loss: 0.227022
Epoch: 3, Step: 19, Loss: 0.231114
Epoch: 3, Step: 20, Loss: 0.260575
Epoch: 3, Step: 20, Loss_val: 0.191757
Epoch: 3, Step: 21, Loss: 0.162695
Epoch: 3, Step:

Epoch: 4, Step: 1, Loss: 0.195405
Epoch: 4, Step: 2, Loss: 0.157378
Epoch: 4, Step: 3, Loss: 0.191871
Epoch: 4, Step: 4, Loss: 0.150633
Epoch: 4, Step: 5, Loss: 0.221148
Epoch: 4, Step: 6, Loss: 0.128144
Epoch: 4, Step: 7, Loss: 0.211611
Epoch: 4, Step: 8, Loss: 0.228441
Epoch: 4, Step: 9, Loss: 0.258749
Epoch: 4, Step: 10, Loss: 0.149385
Epoch: 4, Step: 10, Loss_val: 0.291039
Epoch: 4, Step: 11, Loss: 0.195368
Epoch: 4, Step: 12, Loss: 0.264471
Epoch: 4, Step: 13, Loss: 0.124667
Epoch: 4, Step: 14, Loss: 0.130752
Epoch: 4, Step: 15, Loss: 0.146848
Epoch: 4, Step: 16, Loss: 0.211537
Epoch: 4, Step: 17, Loss: 0.155383
Epoch: 4, Step: 18, Loss: 0.18875
Epoch: 4, Step: 19, Loss: 0.201446
Epoch: 4, Step: 20, Loss: 0.229127
Epoch: 4, Step: 20, Loss_val: 0.260199
Epoch: 4, Step: 21, Loss: 0.195313
Epoch: 4, Step: 22, Loss: 0.273952
Epoch: 4, Step: 23, Loss: 0.311794
Epoch: 4, Step: 24, Loss: 0.139167
Epoch: 4, Step: 25, Loss: 0.175012
Epoch: 4, Step: 26, Loss: 0.182697
Epoch: 4, Step: 27, Lo

Epoch: 5, Step: 3, Loss: 0.165249
Epoch: 5, Step: 4, Loss: 0.132308
Epoch: 5, Step: 5, Loss: 0.197145
Epoch: 5, Step: 6, Loss: 0.238048
Epoch: 5, Step: 7, Loss: 0.164154
Epoch: 5, Step: 8, Loss: 0.243249
Epoch: 5, Step: 9, Loss: 0.165236
Epoch: 5, Step: 10, Loss: 0.172639
Epoch: 5, Step: 10, Loss_val: 0.198632
Epoch: 5, Step: 11, Loss: 0.136421
Epoch: 5, Step: 12, Loss: 0.215613
Epoch: 5, Step: 13, Loss: 0.165157
Epoch: 5, Step: 14, Loss: 0.122606
Epoch: 5, Step: 15, Loss: 0.157107
Epoch: 5, Step: 16, Loss: 0.184407
Epoch: 5, Step: 17, Loss: 0.157959
Epoch: 5, Step: 18, Loss: 0.114995
Epoch: 5, Step: 19, Loss: 0.162902
Epoch: 5, Step: 20, Loss: 0.214435
Epoch: 5, Step: 20, Loss_val: 0.173668
Epoch: 5, Step: 21, Loss: 0.145797
Epoch: 5, Step: 22, Loss: 0.123086
Epoch: 5, Step: 23, Loss: 0.157313
Epoch: 5, Step: 24, Loss: 0.206707
Epoch: 5, Step: 25, Loss: 0.152449
Epoch: 5, Step: 26, Loss: 0.133876
Epoch: 5, Step: 27, Loss: 0.139543
Epoch: 5, Step: 28, Loss: 0.112796
Epoch: 5, Step: 29,

Epoch: 6, Step: 6, Loss: 0.212228
Epoch: 6, Step: 7, Loss: 0.208624
Epoch: 6, Step: 8, Loss: 0.136991
Epoch: 6, Step: 9, Loss: 0.198931
Epoch: 6, Step: 10, Loss: 0.159773
Epoch: 6, Step: 10, Loss_val: 0.277023
Epoch: 6, Step: 11, Loss: 0.156838
Epoch: 6, Step: 12, Loss: 0.101745
Epoch: 6, Step: 13, Loss: 0.137961
Epoch: 6, Step: 14, Loss: 0.156431
Epoch: 6, Step: 15, Loss: 0.117375
Epoch: 6, Step: 16, Loss: 0.220412
Epoch: 6, Step: 17, Loss: 0.142837
Epoch: 6, Step: 18, Loss: 0.129934
Epoch: 6, Step: 19, Loss: 0.140735
Epoch: 6, Step: 20, Loss: 0.160063
Epoch: 6, Step: 20, Loss_val: 0.11585
Epoch: 6, Step: 21, Loss: 0.119935
Epoch: 6, Step: 22, Loss: 0.194459
Epoch: 6, Step: 23, Loss: 0.113538
Epoch: 6, Step: 24, Loss: 0.266716
Epoch: 6, Step: 25, Loss: 0.119688
Epoch: 6, Step: 26, Loss: 0.18723
Epoch: 6, Step: 27, Loss: 0.193336
Epoch: 6, Step: 28, Loss: 0.125692
Epoch: 6, Step: 29, Loss: 0.106951
Epoch: 6, Step: 30, Loss: 0.141298
Epoch: 6, Step: 30, Loss_val: 0.212399
Epoch: 6, Step

Epoch: 7, Step: 9, Loss: 0.12813
Epoch: 7, Step: 10, Loss: 0.124941
Epoch: 7, Step: 10, Loss_val: 0.19299
Epoch: 7, Step: 11, Loss: 0.100701
Epoch: 7, Step: 12, Loss: 0.149139
Epoch: 7, Step: 13, Loss: 0.174274
Epoch: 7, Step: 14, Loss: 0.156149
Epoch: 7, Step: 15, Loss: 0.142964
Epoch: 7, Step: 16, Loss: 0.161576
Epoch: 7, Step: 17, Loss: 0.172429
Epoch: 7, Step: 18, Loss: 0.122951
Epoch: 7, Step: 19, Loss: 0.199801
Epoch: 7, Step: 20, Loss: 0.177917
Epoch: 7, Step: 20, Loss_val: 0.322603
Epoch: 7, Step: 21, Loss: 0.0983639
Epoch: 7, Step: 22, Loss: 0.113613
Epoch: 7, Step: 23, Loss: 0.110209
Epoch: 7, Step: 24, Loss: 0.286777
Epoch: 7, Step: 25, Loss: 0.136909
Epoch: 7, Step: 26, Loss: 0.138857
Epoch: 7, Step: 27, Loss: 0.157705
Epoch: 7, Step: 28, Loss: 0.170651
Epoch: 7, Step: 29, Loss: 0.104289
Epoch: 7, Step: 30, Loss: 0.109843
Epoch: 7, Step: 30, Loss_val: 0.184746
Epoch: 7, Step: 31, Loss: 0.149387
Epoch: 7, Step: 32, Loss: 0.0993512
Epoch: 7, Step: 33, Loss: 0.0940173
Epoch: 7

Epoch: 8, Step: 11, Loss: 0.0859142
Epoch: 8, Step: 12, Loss: 0.110144
Epoch: 8, Step: 13, Loss: 0.15192
Epoch: 8, Step: 14, Loss: 0.100375
Epoch: 8, Step: 15, Loss: 0.0900984
Epoch: 8, Step: 16, Loss: 0.0747057
Epoch: 8, Step: 17, Loss: 0.135452
Epoch: 8, Step: 18, Loss: 0.0587898
Epoch: 8, Step: 19, Loss: 0.0965583
Epoch: 8, Step: 20, Loss: 0.139614
Epoch: 8, Step: 20, Loss_val: 0.0759793
Epoch: 8, Step: 21, Loss: 0.140973
Epoch: 8, Step: 22, Loss: 0.0686802
Epoch: 8, Step: 23, Loss: 0.248053
Epoch: 8, Step: 24, Loss: 0.0912862
Epoch: 8, Step: 25, Loss: 0.0809813
Epoch: 8, Step: 26, Loss: 0.0566642
Epoch: 8, Step: 27, Loss: 0.13382
Epoch: 8, Step: 28, Loss: 0.0708314
Epoch: 8, Step: 29, Loss: 0.108566
Epoch: 8, Step: 30, Loss: 0.115928
Epoch: 8, Step: 30, Loss_val: 0.267804
Epoch: 8, Step: 31, Loss: 0.0451051
Epoch: 8, Step: 32, Loss: 0.0574074
Epoch: 8, Step: 33, Loss: 0.122176
Epoch: 8, Step: 34, Loss: 0.167278
Epoch: 8, Step: 35, Loss: 0.120453
Epoch: 8, Step: 36, Loss: 0.0366684


Epoch: 9, Step: 11, Loss: 0.124389
Epoch: 9, Step: 12, Loss: 0.0794655
Epoch: 9, Step: 13, Loss: 0.131853
Epoch: 9, Step: 14, Loss: 0.088974
Epoch: 9, Step: 15, Loss: 0.0896269
Epoch: 9, Step: 16, Loss: 0.117905
Epoch: 9, Step: 17, Loss: 0.137383
Epoch: 9, Step: 18, Loss: 0.0981222
Epoch: 9, Step: 19, Loss: 0.0960742
Epoch: 9, Step: 20, Loss: 0.13197
Epoch: 9, Step: 20, Loss_val: 0.157616
Epoch: 9, Step: 21, Loss: 0.154027
Epoch: 9, Step: 22, Loss: 0.0447076
Epoch: 9, Step: 23, Loss: 0.11539
Epoch: 9, Step: 24, Loss: 0.109739
Epoch: 9, Step: 25, Loss: 0.193493
Epoch: 9, Step: 26, Loss: 0.0987393
Epoch: 9, Step: 27, Loss: 0.096093
Epoch: 9, Step: 28, Loss: 0.0935864
Epoch: 9, Step: 29, Loss: 0.0630737
Epoch: 9, Step: 30, Loss: 0.195858
Epoch: 9, Step: 30, Loss_val: 0.182695
Epoch: 9, Step: 31, Loss: 0.0945128
Epoch: 9, Step: 32, Loss: 0.20508
Epoch: 9, Step: 33, Loss: 0.10613
Epoch: 9, Step: 34, Loss: 0.14115
Epoch: 9, Step: 35, Loss: 0.107874
Epoch: 9, Step: 36, Loss: 0.137435
Epoch: 9

In [2]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import numpy as np
import time

begin = time.strftime('%Y-%m-%d_%H-%M-%S')

LOGDIR = './save'

tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

start_learning_rate = 0.5e-3  # 1e-3
adjust_learning_rate = 1e-5

onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=4)

loss = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

loss_val = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss_val = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
# train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
# tf.scalar_summary("loss", loss)
tf.summary.scalar("loss", loss)

tf.summary.scalar("loss_val", loss_val)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)
# saver.restore(sess, "save/model.ckpt")

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 10   # 13, 8, 12, 20
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        # train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
        loss_value = loss.eval(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0})
        # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
        print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        if i % 10 == 0:
            xs_val, ys_val = driving_data.LoadValBatch(batch_size)
            # xs, ys = driving_data.LoadTrainBatch(batch_size)
            loss_val = loss.eval(feed_dict={model.x:xs_val, model.y_: ys_val, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss_val: %g" % (epoch, i, loss_val))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)


correct_prediction = tf.equal(tf.argmax(onehot_labels, 1), tf.argmax(model.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Train Accuracy:', sess.run(accuracy, feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}))
print('Validation Accuracy:', sess.run(accuracy, feed_dict={model.x: xs_val, model.y_: ys_val, model.keep_prob: 1.0}))

end = time.strftime('%Y-%m-%d_%H-%M-%S')
print('begin: ', begin)
print('end: ', end)

print("Run the command line:\n",
      "--> tensorboard --logdir=./logs --port=6006",
      "\nThen open http://0.0.0.0:6006/ into your web browser")

# os.system("sudo -s shutdown -h now")


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch: 0, Step: 0, Loss: 1.41853
Epoch: 0, Step: 0, Loss_val: 1.41076
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 1, Loss: 1.38697
Epoch: 0, Step: 2, Loss: 1.37533
Epoch: 0, Step: 3, Loss: 1.3571
Epoch: 0, Step: 4, Loss: 1.34977
Epoch: 0, Step: 5, Loss: 1.35506
Epoch: 0, Step: 6, Loss: 1.34243
Epoch: 0, Step: 7, Loss: 1.35286
Epoch: 0, Step: 8, Loss: 1.32332
Epoch: 0, Step: 9, Loss: 1.32072
Epoch: 0, Step: 10, Loss: 1.35055
Epoch: 0, Step: 10, Loss_val: 1.34358
Epoch: 0, Step: 11, Loss: 1.34976
Epoch: 0, Step: 12, Loss: 1.29859
Epoch: 0, Step: 13, Loss: 1.30691
Epoch: 0, Step: 14, Loss: 1.32576
Epoch: 0, Step: 15, Loss: 1.33193
Epoch: 0, Step: 16, Loss: 1.33941
Epoch: 0, Step: 17, Loss: 1.31562
Epoch: 0, Step: 18, Loss: 1.27066
Epoch: 0, Step: 19, Loss: 1.25146
Epoch: 0, Step: 20, Loss: 1.27585
Epoch: 0, Step: 20, Loss_val: 1.30012
Epoch: 0, Step: 21, Loss: 1.27242
Epoch: 0, Step: 22, Loss: 1.31237
Epoch: 0, Step: 23, Loss: 1.34067
Epoch: 0, Step: 24, Loss: 1.33176
Epoch: 0,

Epoch: 1, Step: 5, Loss: 0.326328
Epoch: 1, Step: 6, Loss: 0.332259
Epoch: 1, Step: 7, Loss: 0.4449
Epoch: 1, Step: 8, Loss: 0.408776
Epoch: 1, Step: 9, Loss: 0.340281
Epoch: 1, Step: 10, Loss: 0.290963
Epoch: 1, Step: 10, Loss_val: 0.345063
Epoch: 1, Step: 11, Loss: 0.35019
Epoch: 1, Step: 12, Loss: 0.273932
Epoch: 1, Step: 13, Loss: 0.397125
Epoch: 1, Step: 14, Loss: 0.439839
Epoch: 1, Step: 15, Loss: 0.388642
Epoch: 1, Step: 16, Loss: 0.344159
Epoch: 1, Step: 17, Loss: 0.454399
Epoch: 1, Step: 18, Loss: 0.439257
Epoch: 1, Step: 19, Loss: 0.312487
Epoch: 1, Step: 20, Loss: 0.400624
Epoch: 1, Step: 20, Loss_val: 0.319332
Epoch: 1, Step: 21, Loss: 0.411642
Epoch: 1, Step: 22, Loss: 0.274803
Epoch: 1, Step: 23, Loss: 0.435475
Epoch: 1, Step: 24, Loss: 0.376724
Epoch: 1, Step: 25, Loss: 0.400227
Epoch: 1, Step: 26, Loss: 0.361322
Epoch: 1, Step: 27, Loss: 0.300517
Epoch: 1, Step: 28, Loss: 0.361342
Epoch: 1, Step: 29, Loss: 0.361946
Epoch: 1, Step: 30, Loss: 0.31823
Epoch: 1, Step: 30, L

Epoch: 2, Step: 10, Loss: 0.373847
Epoch: 2, Step: 10, Loss_val: 0.232761
Epoch: 2, Step: 11, Loss: 0.182127
Epoch: 2, Step: 12, Loss: 0.207653
Epoch: 2, Step: 13, Loss: 0.188726
Epoch: 2, Step: 14, Loss: 0.312001
Epoch: 2, Step: 15, Loss: 0.182091
Epoch: 2, Step: 16, Loss: 0.210909
Epoch: 2, Step: 17, Loss: 0.236305
Epoch: 2, Step: 18, Loss: 0.207181
Epoch: 2, Step: 19, Loss: 0.137831
Epoch: 2, Step: 20, Loss: 0.302713
Epoch: 2, Step: 20, Loss_val: 0.29736
Epoch: 2, Step: 21, Loss: 0.291617
Epoch: 2, Step: 22, Loss: 0.261941
Epoch: 2, Step: 23, Loss: 0.32022
Epoch: 2, Step: 24, Loss: 0.474934
Epoch: 2, Step: 25, Loss: 0.218117
Epoch: 2, Step: 26, Loss: 0.202877
Epoch: 2, Step: 27, Loss: 0.245289
Epoch: 2, Step: 28, Loss: 0.246577
Epoch: 2, Step: 29, Loss: 0.243502
Epoch: 2, Step: 30, Loss: 0.283394
Epoch: 2, Step: 30, Loss_val: 0.402878
Epoch: 2, Step: 31, Loss: 0.312725
Epoch: 2, Step: 32, Loss: 0.268362
Epoch: 2, Step: 33, Loss: 0.214608
Epoch: 2, Step: 34, Loss: 0.465917
Epoch: 2, 

Epoch: 3, Step: 13, Loss: 0.234959
Epoch: 3, Step: 14, Loss: 0.271018
Epoch: 3, Step: 15, Loss: 0.26344
Epoch: 3, Step: 16, Loss: 0.210031
Epoch: 3, Step: 17, Loss: 0.275292
Epoch: 3, Step: 18, Loss: 0.1602
Epoch: 3, Step: 19, Loss: 0.19885
Epoch: 3, Step: 20, Loss: 0.282407
Epoch: 3, Step: 20, Loss_val: 0.391356
Epoch: 3, Step: 21, Loss: 0.326067
Epoch: 3, Step: 22, Loss: 0.202224
Epoch: 3, Step: 23, Loss: 0.174241
Epoch: 3, Step: 24, Loss: 0.210862
Epoch: 3, Step: 25, Loss: 0.2514
Epoch: 3, Step: 26, Loss: 0.305439
Epoch: 3, Step: 27, Loss: 0.181868
Epoch: 3, Step: 28, Loss: 0.167579
Epoch: 3, Step: 29, Loss: 0.216151
Epoch: 3, Step: 30, Loss: 0.136393
Epoch: 3, Step: 30, Loss_val: 0.232937
Epoch: 3, Step: 31, Loss: 0.224004
Epoch: 3, Step: 32, Loss: 0.184322
Epoch: 3, Step: 33, Loss: 0.280744
Epoch: 3, Step: 34, Loss: 0.180434
Epoch: 3, Step: 35, Loss: 0.259609
Epoch: 3, Step: 36, Loss: 0.225523
Epoch: 3, Step: 37, Loss: 0.26079
Epoch: 3, Step: 38, Loss: 0.264125
Epoch: 3, Step: 39,

...

Epoch: 5, Step: 21, Loss: 0.231267
Epoch: 5, Step: 22, Loss: 0.190895
Epoch: 5, Step: 23, Loss: 0.0636059
Epoch: 5, Step: 24, Loss: 0.193496
Epoch: 5, Step: 25, Loss: 0.223624
Epoch: 5, Step: 26, Loss: 0.313194
Epoch: 5, Step: 27, Loss: 0.115757
Epoch: 5, Step: 28, Loss: 0.192619
Epoch: 5, Step: 29, Loss: 0.244828
Epoch: 5, Step: 30, Loss: 0.16427
Epoch: 5, Step: 30, Loss_val: 0.207845
Epoch: 5, Step: 31, Loss: 0.151505
Epoch: 5, Step: 32, Loss: 0.142915
Epoch: 5, Step: 33, Loss: 0.202138
Epoch: 5, Step: 34, Loss: 0.125827
Epoch: 5, Step: 35, Loss: 0.110604
Epoch: 5, Step: 36, Loss: 0.128189
Epoch: 5, Step: 37, Loss: 0.134716
Epoch: 5, Step: 38, Loss: 0.102132
Epoch: 5, Step: 39, Loss: 0.113404
Epoch: 5, Step: 40, Loss: 0.21188
Epoch: 5, Step: 40, Loss_val: 0.361467
Epoch: 5, Step: 41, Loss: 0.217607
Epoch: 5, Step: 42, Loss: 0.166851
Epoch: 5, Step: 43, Loss: 0.252742
Epoch: 5, Step: 44, Loss: 0.157131
Epoch: 5, Step: 45, Loss: 0.104298
Epoch: 5, Step: 46, Loss: 0.119825
Epoch: 5, Ste

Epoch: 6, Step: 25, Loss: 0.225193
Epoch: 6, Step: 26, Loss: 0.168292
Epoch: 6, Step: 27, Loss: 0.100257
Epoch: 6, Step: 28, Loss: 0.138608
Epoch: 6, Step: 29, Loss: 0.102497
Epoch: 6, Step: 30, Loss: 0.164978
Epoch: 6, Step: 30, Loss_val: 0.26354
Epoch: 6, Step: 31, Loss: 0.129643
Epoch: 6, Step: 32, Loss: 0.151103
Epoch: 6, Step: 33, Loss: 0.102521
Epoch: 6, Step: 34, Loss: 0.119766
Epoch: 6, Step: 35, Loss: 0.172627
Epoch: 6, Step: 36, Loss: 0.119597
Epoch: 6, Step: 37, Loss: 0.185342
Epoch: 6, Step: 38, Loss: 0.137785
Epoch: 6, Step: 39, Loss: 0.110987
Epoch: 6, Step: 40, Loss: 0.132947
Epoch: 6, Step: 40, Loss_val: 0.185882
Epoch: 6, Step: 41, Loss: 0.162476
Epoch: 6, Step: 42, Loss: 0.158942
Epoch: 6, Step: 43, Loss: 0.11539
Epoch: 6, Step: 44, Loss: 0.162595
Epoch: 6, Step: 45, Loss: 0.142872
Epoch: 6, Step: 46, Loss: 0.148209
Epoch: 6, Step: 47, Loss: 0.150558
Epoch: 6, Step: 48, Loss: 0.239382
Epoch: 6, Step: 49, Loss: 0.177109
Epoch: 6, Step: 50, Loss: 0.123153
Epoch: 6, Step

Epoch: 7, Step: 28, Loss: 0.104891
Epoch: 7, Step: 29, Loss: 0.112649
Epoch: 7, Step: 30, Loss: 0.103768
Epoch: 7, Step: 30, Loss_val: 0.462116
Epoch: 7, Step: 31, Loss: 0.121403
Epoch: 7, Step: 32, Loss: 0.128071
Epoch: 7, Step: 33, Loss: 0.124207
Epoch: 7, Step: 34, Loss: 0.208873
Epoch: 7, Step: 35, Loss: 0.103509
Epoch: 7, Step: 36, Loss: 0.145316
Epoch: 7, Step: 37, Loss: 0.111616
Epoch: 7, Step: 38, Loss: 0.201875
Epoch: 7, Step: 39, Loss: 0.240398
Epoch: 7, Step: 40, Loss: 0.102243
Epoch: 7, Step: 40, Loss_val: 0.0998037
Epoch: 7, Step: 41, Loss: 0.0621247
Epoch: 7, Step: 42, Loss: 0.0970413
Epoch: 7, Step: 43, Loss: 0.190539
Epoch: 7, Step: 44, Loss: 0.0636678
Epoch: 7, Step: 45, Loss: 0.130366
Epoch: 7, Step: 46, Loss: 0.103184
Epoch: 7, Step: 47, Loss: 0.0935201
Epoch: 7, Step: 48, Loss: 0.157346
Epoch: 7, Step: 49, Loss: 0.227672
Epoch: 7, Step: 50, Loss: 0.142659
Epoch: 7, Step: 50, Loss_val: 0.124077
Epoch: 7, Step: 51, Loss: 0.114179
Epoch: 7, Step: 52, Loss: 0.253589
Epo

Epoch: 8, Step: 31, Loss: 0.129102
Epoch: 8, Step: 32, Loss: 0.0965987
Epoch: 8, Step: 33, Loss: 0.123393
Epoch: 8, Step: 34, Loss: 0.092445
Epoch: 8, Step: 35, Loss: 0.139873
Epoch: 8, Step: 36, Loss: 0.156561
Epoch: 8, Step: 37, Loss: 0.133696
Epoch: 8, Step: 38, Loss: 0.174836
Epoch: 8, Step: 39, Loss: 0.141737
Epoch: 8, Step: 40, Loss: 0.0729809
Epoch: 8, Step: 40, Loss_val: 0.137865
Epoch: 8, Step: 41, Loss: 0.116107
Epoch: 8, Step: 42, Loss: 0.127157
Epoch: 8, Step: 43, Loss: 0.106585
Epoch: 8, Step: 44, Loss: 0.117147
Epoch: 8, Step: 45, Loss: 0.121752
Epoch: 8, Step: 46, Loss: 0.0640129
Epoch: 8, Step: 47, Loss: 0.210707
Epoch: 8, Step: 48, Loss: 0.091275
Epoch: 8, Step: 49, Loss: 0.117247
Epoch: 8, Step: 50, Loss: 0.0830273
Epoch: 8, Step: 50, Loss_val: 0.0615793
Epoch: 8, Step: 51, Loss: 0.136782
Epoch: 8, Step: 52, Loss: 0.0899698
Epoch: 8, Step: 53, Loss: 0.12419
Epoch: 8, Step: 54, Loss: 0.0576245
Epoch: 8, Step: 55, Loss: 0.155935
Epoch: 8, Step: 56, Loss: 0.129172
Epoch:

Epoch: 9, Step: 32, Loss: 0.0630684
Epoch: 9, Step: 33, Loss: 0.0725563
Epoch: 9, Step: 34, Loss: 0.119207
Epoch: 9, Step: 35, Loss: 0.0511642
Epoch: 9, Step: 36, Loss: 0.0726704
Epoch: 9, Step: 37, Loss: 0.111923
Epoch: 9, Step: 38, Loss: 0.0893198
Epoch: 9, Step: 39, Loss: 0.0801693
Epoch: 9, Step: 40, Loss: 0.0691854
Epoch: 9, Step: 40, Loss_val: 0.065739
Epoch: 9, Step: 41, Loss: 0.166553
Epoch: 9, Step: 42, Loss: 0.100895
Epoch: 9, Step: 43, Loss: 0.0764726
Epoch: 9, Step: 44, Loss: 0.229151
Epoch: 9, Step: 45, Loss: 0.110387
Epoch: 9, Step: 46, Loss: 0.0688456
Epoch: 9, Step: 47, Loss: 0.0832194
Epoch: 9, Step: 48, Loss: 0.0648649
Epoch: 9, Step: 49, Loss: 0.118627
Epoch: 9, Step: 50, Loss: 0.122248
Epoch: 9, Step: 50, Loss_val: 0.224639
Epoch: 9, Step: 51, Loss: 0.140869
Epoch: 9, Step: 52, Loss: 0.173301
Epoch: 9, Step: 53, Loss: 0.11465
Epoch: 9, Step: 54, Loss: 0.063516
Epoch: 9, Step: 55, Loss: 0.151783
Epoch: 9, Step: 56, Loss: 0.0940225
Epoch: 9, Step: 57, Loss: 0.0817087
